In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

In [3]:
def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= point:
            print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

def reindex(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    span = 0
    for i in range (len(Stock_list)):
        if  Stock_list[i] == extract.iloc[i-span][label]:
            institutional=institutional.append(extract[i-span:i-span+1])
        else :
            print (i,span , Stock_list[i], extract.iloc[i-span][label])
            ## if extract has stock which base don't have
            if  extract.iloc[i-span][label] not in Stock_list.unique():
                span -= 1
                if Stock_list[i] == extract.iloc[i-span][label]:
                    institutional=institutional.append(extract[i-span:i-span+1])
                else:
                    span += 1
                    institutional=institutional.append(null)
                    
            else:    
                institutional=institutional.append(null)
                span += 1
    return institutional.reset_index(drop=True)

In [123]:
def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == Stock_list[i]].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

In [5]:
def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

In [95]:
datestr = '20210120'

# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")),  header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
# 整理一些字串：
df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"30,367,315","25,701","4,202,770,386",138.10,139.50,137.65,138.20,+,0.25,138.20,7,138.25,63,0.00
1,0051,元大中型100,"56,966",62,"2,530,796",45.20,45.20,43.98,44.05,-,1.10,44.05,2,44.07,14,0.00
2,0052,富邦科技,"2,347,344","2,020","306,078,051",129.20,131.85,129.10,130.65,+,2.15,130.50,2,130.65,16,0.00
3,0053,元大電子,"53,625",57,"3,550,965",65.60,66.75,65.60,65.90,+,0.30,65.95,53,66.35,1,0.00
4,0054,元大台商50,"20,001",12,"603,729",30.54,30.54,30.00,30.00,-,0.61,29.80,52,30.00,2,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21161,9944,新麗,"446,476",255,"9,664,101",21.65,21.90,21.40,21.50,-,0.20,21.50,3,21.55,20,63.24
21162,9945,潤泰新,"7,555,581","4,490","298,781,011",40.00,40.10,39.20,39.35,-,0.65,39.35,49,39.40,88,8.52
21163,9946,三發地產,"610,943",346,"8,809,151",14.65,14.70,14.25,14.50,-,0.40,14.30,1,14.50,18,8.29
21164,9955,佳龍,"611,093",383,"11,065,572",18.55,18.55,17.90,18.00,-,0.55,18.00,6,18.05,7,0.00


In [26]:
stock_df = df[20190:]
stock_df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
20190,1102,亞泥,"8,679,192","4,506","358,687,006",41.85,41.90,41.05,41.45,-,0.70,41.40,1,41.45,25,9.71,NaN
20191,1103,嘉泥,"1,738,385",680,"29,105,621",17.00,17.05,16.60,16.70,-,0.45,16.70,12,16.75,3,31.51,NaN
20192,1104,環泥,"1,641,890",769,"35,310,301",21.95,21.95,21.20,21.25,-,0.90,21.25,50,21.30,28,10.62,NaN
20193,1108,幸福,"1,951,811",536,"20,735,712",10.95,10.95,10.45,10.65,-,0.35,10.60,10,10.65,8,11.58,NaN
20194,1109,信大,"816,320",367,"15,092,253",18.90,18.90,18.25,18.35,-,0.55,18.35,1,18.40,6,6.87,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21161,9944,新麗,"446,476",255,"9,664,101",21.65,21.90,21.40,21.50,-,0.20,21.50,3,21.55,20,63.24,NaN
21162,9945,潤泰新,"7,555,581","4,490","298,781,011",40.00,40.10,39.20,39.35,-,0.65,39.35,49,39.40,88,8.52,NaN
21163,9946,三發地產,"610,943",346,"8,809,151",14.65,14.70,14.25,14.50,-,0.40,14.30,1,14.50,18,8.29,NaN
21164,9955,佳龍,"611,093",383,"11,065,572",18.55,18.55,17.90,18.00,-,0.55,18.00,6,18.05,7,0.00,NaN


In [137]:
stock_df = stock_df.drop(['Unnamed: 16'],axis = 1)
stock_df = stock_df.dropna().reset_index(drop=True)
label_ = "證券代號"

In [138]:
stock_df = check_symbol(stock_df,label_ )

In [145]:
Stock_list = stock_df[label_ ]

In [146]:
Stock_list

0      1102
1      1103
2      1104
3      1108
4      1109
       ... 
914    9944
915    9945
916    9946
917    9955
918    9958
Name: 證券代號, Length: 919, dtype: object

In [35]:
r = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=' + datestr + '&selectType=ALL')
# 整理資料，變成表格
r = r.text.replace("=", "")
df2 = pd.read_csv(StringIO(r.replace("  \n", "")),  header=["證券代號" in l for l in r.split("\r\n")].index(True))

df2.sort_values(by='證券代號' , ascending= True, axis =0 , inplace=True)

df2 = df2.reset_index(drop=True).drop(['Unnamed: 19'],axis = 1)
df2


,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
0,0050,元大台灣50,"15,691,525","14,356,000","1,335,525",0,0,0,"131,000",0,"131,000","-2,138,744","1,591,000","1,736,315","-145,315","2,071,454","4,064,883","-1,993,429","-672,219"
1,0051,元大中型100,"6,000","7,000","-1,000",0,0,0,0,0,0,"-34,472",0,0,0,0,"34,472","-34,472","-35,472"
2,0052,富邦科技,"79,000","79,000",0,0,0,0,"15,000",0,"15,000","-1,192,696",0,0,0,"371,328","1,564,024","-1,192,696","-1,177,696"
3,0053,元大電子,"1,000",0,"1,000",0,0,0,0,0,0,"-24,000",0,0,0,"4,000","28,000","-24,000","-23,000"
4,0054,元大台商50,0,0,0,0,0,0,0,0,0,"3,000",0,0,0,"11,000","8,000","3,000","3,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11333,投信表示本國投資信託基金。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11334,本統計資訊含一般、零股、盤後定價、鉅額，不含拍賣、標購。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11335,本資訊以當日原始成交情形統計，不以證券商申報錯帳、更正帳號等調整後資料統計。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11336,自營商表示證券自營商專戶。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


institutional = pd.DataFrame()
for s in range (len(df2)):
    if df2.iloc[s]['證券代號'] in Stock_list.unique():
        institutional=institutional.append(df2.iloc[s])

In [93]:
label_ = "證券代號"
test = extract(Stock_list , df2 , label_)
test = check_symbol(test,label_)
test

1102 10374
9958 11329


,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
0,1102,亞泥,"1,689,618","5,902,652","-4,213,034",0,0,0,"5,000","510,000","-505,000","444,000","361,000","116,000","245,000","201,000","2,000","199,000","-4,274,034"
1,1103,嘉泥,"194,000","469,000","-275,000",0,0,0,0,0,0,"-9,000","1,000","10,000","-9,000",0,0,0,"-284,000"
2,1104,環泥,"95,000","740,000","-645,000",0,0,0,0,0,0,"-4,906","5,094","10,000","-4,906",0,0,0,"-649,906"
3,1108,幸福,"159,000","536,000","-377,000",0,0,0,0,0,0,"4,000","4,000",0,"4,000",0,0,0,"-373,000"
4,1109,信大,"42,000","268,000","-226,000",0,0,0,0,0,0,0,0,0,0,0,0,0,"-226,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,9944,新麗,"56,000","97,000","-41,000",0,0,0,0,0,0,"3,000","3,000",0,"3,000",0,0,0,"-38,000"
930,9945,潤泰新,"480,643","2,978,500","-2,497,857",0,0,0,0,0,0,"-22,000","23,000","40,000","-17,000","2,000","7,000","-5,000","-2,519,857"
931,9946,三發地產,"27,000","200,000","-173,000",0,0,0,0,0,0,"5,000","5,000",0,"5,000",0,0,0,"-168,000"
932,9955,佳龍,"29,000","169,000","-140,000",0,0,0,0,0,0,0,0,0,0,0,0,0,"-140,000"


In [77]:
institutional = find_index(Stock_list, test, label_,  df2[-1:])
institutional

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
0,1102,亞泥,"1,689,618","5,902,652","-4,213,034",0,0,0,"5,000","510,000","-505,000","444,000","361,000","116,000","245,000","201,000","2,000","199,000","-4,274,034"
1,1103,嘉泥,"194,000","469,000","-275,000",0,0,0,0,0,0,"-9,000","1,000","10,000","-9,000",0,0,0,"-284,000"
2,1104,環泥,"95,000","740,000","-645,000",0,0,0,0,0,0,"-4,906","5,094","10,000","-4,906",0,0,0,"-649,906"
3,1108,幸福,"159,000","536,000","-377,000",0,0,0,0,0,0,"4,000","4,000",0,"4,000",0,0,0,"-373,000"
4,1109,信大,"42,000","268,000","-226,000",0,0,0,0,0,0,0,0,0,0,0,0,0,"-226,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,9944,新麗,"56,000","97,000","-41,000",0,0,0,0,0,0,"3,000","3,000",0,"3,000",0,0,0,"-38,000"
945,9945,潤泰新,"480,643","2,978,500","-2,497,857",0,0,0,0,0,0,"-22,000","23,000","40,000","-17,000","2,000","7,000","-5,000","-2,519,857"
946,9946,三發地產,"27,000","200,000","-173,000",0,0,0,0,0,0,"5,000","5,000",0,"5,000",0,0,0,"-168,000"
947,9955,佳龍,"29,000","169,000","-140,000",0,0,0,0,0,0,0,0,0,0,0,0,0,"-140,000"


In [156]:
r = requests.get('https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' + datestr + '&selectType=ALL')
# 整理資料，變成表格
r = r.text.replace("=", "")
df3 = pd.read_csv(StringIO(r), header=["股票代號" in l for l in r.split("\n")].index(True))
df3 = df3.drop(['Unnamed: 16'],axis = 1)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))

for i in range (947) :
    if margin.iloc[i]["股票代號"] != Stock_list[i] :
        print (margin.iloc[i]["股票代號"], Stock_list[i], i)
test2.iloc[625][label_] not in Stock_list.unique()        

In [116]:
test2 

,股票代號,股票名稱,買進,賣出,現金償還,前日餘額,今日餘額,限額,買進.1,賣出.1,現券償還,前日餘額.1,今日餘額.1,限額.1,資券互抵,註記
118,1102,亞泥,99,24,0,"2,471","2,546","840,361",10,1,0,45,36,"840,361",0,
119,1103,嘉泥,47,206,4,"2,936","2,773","193,695",1,4,0,1,4,"193,695",3,
120,1104,環泥,87,53,0,"1,830","1,864","163,402",1,0,0,15,14,"163,402",0,
121,1108,幸福,609,322,0,"6,213","6,500","101,184",0,2,0,16,18,"101,184",0,
122,1109,信大,18,30,0,543,531,"85,489",0,0,0,9,9,"85,489",0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,9944,新麗,29,14,0,"10,562","10,577","27,276",0,0,0,20,20,"27,276",0,
1056,9945,潤泰新,197,373,0,"8,342","8,166","376,220",0,14,0,549,563,"376,220",0,
1057,9946,三發地產,5,9,0,"1,079","1,075","80,250",1,15,0,9,23,"80,250",0,
1058,9955,佳龍,70,70,0,"1,527","1,527","25,802",0,2,0,1,3,"25,802",0,


In [119]:
test2[825:900]

,股票代號,股票名稱,買進,賣出,現金償還,前日餘額,今日餘額,限額,買進.1,賣出.1,現券償還,前日餘額.1,今日餘額.1,限額.1,資券互抵,註記
943,6592,和潤企業,15,9,1,895,900,"128,750",0,0,0,0,0,"128,750",0,
944,6605,帝寶,64,159,0,"1,493","1,398","41,454",4,4,0,23,23,"41,454",2,
945,6625,必應,0,8,0,97,89,"11,218",0,0,0,2,2,"11,218",0,
946,6641,基士德-KY,0,0,0,33,33,"8,500",0,0,0,0,0,"8,500",0,
947,6655,科定,0,2,0,39,37,"18,902",0,0,0,0,0,"18,902",0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,910322,康師傅-DR,0,0,0,0,0,0,0,0,0,0,0,0,0,OX
1014,9105,泰金寶-DR,569,"1,218",0,"24,132","23,483","209,821",294,6,0,"12,211","11,923","209,821",41,
1015,910861,神州-DR,6,39,0,"1,999","1,966","15,194",0,0,0,85,85,"15,194",0,
1016,911616,杜康-DR,24,13,0,"5,392","5,403","32,500",0,0,0,393,393,"32,500",0,


In [124]:
label_ = "股票代號"
test2 = extract(Stock_list, df3 , label_).reset_index(drop=True)
margin = find_index(Stock_list, test2, label_,  df3[-1:])
margin

1102 118
9958 1059


,股票代號,股票名稱,買進,賣出,現金償還,前日餘額,今日餘額,限額,買進.1,賣出.1,現券償還,前日餘額.1,今日餘額.1,限額.1,資券互抵,註記
0,1102,亞泥,99,24,0,"2,471","2,546","840,361",10,1,0,45,36,"840,361",0,
1,1103,嘉泥,47,206,4,"2,936","2,773","193,695",1,4,0,1,4,"193,695",3,
2,1104,環泥,87,53,0,"1,830","1,864","163,402",1,0,0,15,14,"163,402",0,
3,1108,幸福,609,322,0,"6,213","6,500","101,184",0,2,0,16,18,"101,184",0,
4,1109,信大,18,30,0,543,531,"85,489",0,0,0,9,9,"85,489",0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,9944,新麗,29,14,0,"10,562","10,577","27,276",0,0,0,20,20,"27,276",0,
945,9945,潤泰新,197,373,0,"8,342","8,166","376,220",0,14,0,549,563,"376,220",0,
946,9946,三發地產,5,9,0,"1,079","1,075","80,250",1,15,0,9,23,"80,250",0,
947,9955,佳龍,70,70,0,"1,527","1,527","25,802",0,2,0,1,3,"25,802",0,


In [40]:
ratio = devide(margin['今日餘額.1'], margin['今日餘額'] )
margin ['ratio'] = ratio 

In [41]:
stock_df['融資買'] = margin['買進']
stock_df['融資賣'] = margin['賣出']
stock_df['融資餘'] = margin['今日餘額']
stock_df['融卷買'] = margin['買進.1']
stock_df['融卷賣'] = margin['賣出.1']
stock_df['融卷餘'] = margin['今日餘額.1']
stock_df['ratio'] = margin ['ratio']

stock_df['外資進出'] = institutional['外陸資買賣超股數(不含外資自營商)']
stock_df['外資進'] = institutional['外陸資買進股數(不含外資自營商)']
stock_df['外資出'] = institutional['外陸資賣出股數(不含外資自營商)']
stock_df['投信進出'] = institutional['投信買賣超股數']
stock_df['投信進'] = institutional['投信買進股數']
stock_df['投信出'] = institutional['投信賣出股數']
stock_df['自營進出'] = institutional['自營商買賣超股數(自行買賣)']
stock_df['自營進'] = institutional['自營商買進股數(自行買賣)']
stock_df['自營出'] = institutional['自營商賣出股數(自行買賣)']
stock_df['避險進出'] = institutional['自營商買賣超股數(避險)']
stock_df['避險進'] = institutional['自營商買進股數(避險)']
stock_df['避險出'] = institutional['自營商賣出股數(避險)']
stock_df['三大'] = institutional['三大法人買賣超股數']

In [42]:
stock_df.columns

Index(['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價',
       '漲跌(+/-)', '漲跌價差', '最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比', '融資買',
       '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進', '外資出',
       '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出', '三大'],
      dtype='object')

In [43]:
stock_df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),...,投信進出,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大
0,1102,亞泥,"8,679,192","4,506","358,687,006",41.85,41.90,41.05,41.45,-,...,"-505,000","5,000","510,000","245,000","361,000","116,000","199,000","201,000","2,000","-4,274,034"
1,1103,嘉泥,"1,738,385",680,"29,105,621",17.00,17.05,16.60,16.70,-,...,0,0,0,"-9,000","1,000","10,000",0,0,0,"-284,000"
2,1104,環泥,"1,641,890",769,"35,310,301",21.95,21.95,21.20,21.25,-,...,0,0,0,"-4,906","5,094","10,000",0,0,0,"-649,906"
3,1108,幸福,"1,951,811",536,"20,735,712",10.95,10.95,10.45,10.65,-,...,0,0,0,"4,000","4,000",0,0,0,0,"-373,000"
4,1109,信大,"816,320",367,"15,092,253",18.90,18.90,18.25,18.35,-,...,0,0,0,0,0,0,0,0,0,"-226,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,9944,新麗,"446,476",255,"9,664,101",21.65,21.90,21.40,21.50,-,...,0,0,0,"3,000","3,000",0,0,0,0,"-38,000"
945,9945,潤泰新,"7,555,581","4,490","298,781,011",40.00,40.10,39.20,39.35,-,...,0,0,0,"-17,000","23,000","40,000","-5,000","2,000","7,000","-2,519,857"
946,9946,三發地產,"610,943",346,"8,809,151",14.65,14.70,14.25,14.50,-,...,0,0,0,"5,000","5,000",0,0,0,0,"-168,000"
947,9955,佳龍,"611,093",383,"11,065,572",18.55,18.55,17.90,18.00,-,...,0,0,0,0,0,0,0,0,0,"-140,000"


In [79]:
date = np.zeros(len(stock_df))

date = ["20200121" for i in range(len(stock_df))]

In [89]:
stock_df ["date"] = date

In [91]:
stock_df  = stock_df.dropna().reset_index(drop=True)

In [97]:
save = pd.DataFrame()
save ["name"] = stock_df["證券代號"]
save.to_csv('name.csv')

In [149]:
test = pd.read_csv('name.csv')
Stock_list = test["name"]

In [150]:
Stock_list[1] in stock_df["證券代號"]

False

In [153]:
Stock_list

0      1102
1      1103
2      1104
3      1108
4      1109
       ... 
914    9944
915    9945
916    9946
917    9955
918    9958
Name: name, Length: 919, dtype: int64